In [12]:
import os
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import torch
import torchvision.transforms.functional as TF
import tifffile
import yaml
from einops import rearrange


In [13]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
%autoreload 2

In [15]:
from mushroom.data.multiplex import extract_ome_tiff, write_basic_ome_tiff, get_ome_tiff_channels, make_pseudo
from mushroom.data.visium import adata_from_visium
from mushroom.visualization.utils import display_sections
import mushroom.utils as utils

In [16]:
run_dir = '/data/estorrs/mushroom/data/projects/xenium_grant'
mushroom_dir = os.path.join(run_dir, 'mushroom')
Path(mushroom_dir).mkdir(parents=True, exist_ok=True)

In [24]:
metadata = yaml.safe_load(open(os.path.join(run_dir, 'registered', 'metadata.yaml')))
metadata

{'HT206B1': {'data': {'multiplex': {'HT206B1-U10': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s5_multiplex.ome.tiff',
    'HT206B1-U13': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s6_multiplex.ome.tiff',
    'HT206B1-U18': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s9_multiplex.ome.tiff',
    'HT206B1-U2': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s1_multiplex.ome.tiff',
    'HT206B1-U21': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s10_multiplex.ome.tiff',
    'HT206B1-U5': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s2_multiplex.ome.tiff'},
   'xenium': {'HT206B1-U1': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s0_xenium.h5ad',
    'HT206B1-U16': '/diskmnt/Projects/Users/estor

In [25]:
configs = {}
for case, data in metadata.items():
    config = [
        {
            'id': sid,
            'position': 0,
            'data': []
        }
        for sid in data['ids']
    ]
    
    for dtype, mapping in data['data'].items():
        for sample_id, filepath in mapping.items():
            idx = data['order'].index(sample_id)
            
            # convert from katmai to glacier
            filepath = filepath.replace('/diskmnt/Projects/Users/estorrs', '/data/estorrs')
            
            config[idx]['data'].append(
                {
                    'dtype': dtype,
                    'filepath': filepath,
                }
            )
    configs[case] = config
configs
    

{'HT206B1': [{'id': 's0',
   'position': 0,
   'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s0_xenium.h5ad'}]},
  {'id': 's1',
   'position': 0,
   'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s1_multiplex.ome.tiff'}]},
  {'id': 's2',
   'position': 0,
   'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s2_multiplex.ome.tiff'}]},
  {'id': 's3',
   'position': 0,
   'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s3_xenium.h5ad'}]},
  {'id': 's4',
   'position': 0,
   'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s4_xenium.h5ad'}]},
  {'id': 's5',
   'position': 0,
   'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/

In [26]:
import sys
yaml.safe_dump(configs, sys.stdout)

HT206B1:
- data:
  - dtype: xenium
    filepath: /data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s0_xenium.h5ad
  id: s0
  position: 0
- data:
  - dtype: multiplex
    filepath: /data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s1_multiplex.ome.tiff
  id: s1
  position: 0
- data:
  - dtype: multiplex
    filepath: /data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s2_multiplex.ome.tiff
  id: s2
  position: 0
- data:
  - dtype: xenium
    filepath: /data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s3_xenium.h5ad
  id: s3
  position: 0
- data:
  - dtype: xenium
    filepath: /data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s4_xenium.h5ad
  id: s4
  position: 0
- data:
  - dtype: multiplex
    filepath: /data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s5_multiplex.ome.tiff
  id: s5
  position: 0
- data:
  - dtype: multiplex
    filepath: /data/estorrs/mushroom/data/projects

In [19]:
# # filling out position seperately and reading back in
# yaml.safe_dump(configs, open(os.path.join(mushroom_dir, 'configs_template.yaml'), 'w'))

In [28]:
configs = yaml.safe_load(open(os.path.join(mushroom_dir, 'configs_template.yaml')))
configs

{'HT206B1': [{'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s0_xenium.h5ad'}],
   'id': 's0',
   'position': 0},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s1_multiplex.ome.tiff'}],
   'id': 's1',
   'position': 5},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s2_multiplex.ome.tiff'}],
   'id': 's2',
   'position': 20},
  {'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s3_xenium.h5ad'}],
   'id': 's3',
   'position': 35},
  {'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/xenium_grant/registered/HT206B1/s4_xenium.h5ad'}],
   'id': 's4',
   'position': 40},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/xenium_grant/reg

In [29]:
for case, config in configs.items():
    directory = os.path.join(mushroom_dir, case)
    Path(directory).mkdir(parents=True, exist_ok=True)
    
    yaml.safe_dump(config, open(os.path.join(directory, 'config.yaml'), 'w'))

In [30]:
mushroom_dir

'/data/estorrs/mushroom/data/projects/xenium_grant/mushroom'